In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
import pickle
import json
import h5py
import pandas as pd


cuda = torch.device('cuda')
frame_size = 10
movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
# on my system os error often pops up
# try to open the file explorer and copy paste the path again 
f = h5py.File("../data/static_ml20m.hdf5", "r")
meta = json.load(open('../data/metadata.json'))

In [3]:
class Actor(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=2e-1):
        super(Actor, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        # state = self.state_rep(state)
        x = F.relu(self.linear1(state))
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        x = self.drop_layer(x)
        # x = torch.tanh(self.linear3(x)) # in case embeds are -1 1 normalized
        x = self.linear3(x) # in case embeds are standard scaled / wiped using PCA whitening
        # return state, x
        return x

In [4]:
ddpg = Actor(1290, 128, 256)
td3 = Actor(1290, 128, 256)
ddpg.load_state_dict(torch.load('../models/DDPG/ddpg_policy(Radam).pt'))
td3.load_state_dict(torch.load('../models/TD3/td3_policy_1.pt'))

batch_size = 5000
n_batches = (f['done'].shape[0] // batch_size) - 1

In [5]:
test_batch = [f[key][n_batches*batch_size:(n_batches+1)*batch_size] for key in
                 ['movies', 'ratings', 'done']]


movies, ratings, done = test_batch
movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
state = torch.cat([movies_tensor[:, :-1, :].view(batch_size, -1),
                       ratings[:, :-1]], 1)


In [6]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}

In [7]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [40]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [41]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,117125,11.480760,The Man Who Loved Yngve,Drama,no,2008,6.3
1,117061,11.713317,The Green,Drama,en,2011,6.2
2,78211,11.803260,Detour,Horror Thriller,no,2009,5.3
3,83437,11.959196,The Young in Heart,Comedy,en,1938,6.2
4,91048,12.021422,Cherry,Comedy Drama Romance,en,2010,6.5
5,40969,12.052855,First Descent,Documentary,en,2005,6.0
6,80844,12.075539,Lake Mungo,Horror Thriller,en,2008,6.2
7,94192,12.107853,Devil's Diary,Drama Horror,en,2007,4.5
8,73475,12.122259,Semper Fi,Action Drama,en,2001,0.0
9,7213,12.166412,Divorce American Style,Comedy,en,1967,6.3


In [42]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,78211,0.602727,Detour,Horror Thriller,no,2009,5.3
1,8825,0.653807,Cheerleader Camp,Comedy Horror Thriller,en,1988,5.0
2,87279,0.662637,Brotherhood,Drama,da,2009,7.5
3,110058,0.662637,Brotherhood,Drama,da,2009,7.5
4,127250,0.675282,Micke & Veronica,Comedy,sv,2014,5.6
5,5751,0.678110,Goodbye Pork Pie,Adventure Comedy,en,1981,6.8
6,4159,0.681150,3000 Miles to Graceland,Action Adventure Comedy Crime Thriller,en,2001,5.7
7,73475,0.684284,Semper Fi,Action Drama,en,2001,0.0
8,47626,0.685158,San Quentin,Action Drama,en,1937,6.2
9,7345,0.686007,Agent Cody Banks 2: Destination London,Comedy Action Adventure Family,en,2004,5.1


In [31]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,87327,0.656696,Camp de Thiaroye,War Drama,fr,1988,6.5
1,5139,0.661534,The Bad News Bears,Comedy Family,en,1976,7.0
2,94539,0.666333,Viva Riva!,Action Drama Thriller,fr,2010,5.7
3,105157,0.677682,Supporting Characters,Drama Comedy,en,2012,6.7
4,7702,0.678041,The Bells of St. Mary's,Drama Family,en,1945,7.2
5,55668,0.678840,Full of It,Comedy Family Drama,en,2007,5.3
6,59382,0.680567,Flight of the Red Balloon,Family Drama,fr,2007,6.0
7,90201,0.683273,The Grand Dukes,Comedy,fr,1996,6.3
8,3611,0.685896,Saludos Amigos,Animation Family,en,1942,6.0
9,4054,0.686331,Save the Last Dance,Drama Family Romance Musical,en,2001,6.4


In [32]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,59382,73.048688,Flight of the Red Balloon,Family Drama,fr,2007,6.0
1,78606,76.857423,Don't Look Back,Drama Horror Mystery Thriller,fr,2009,5.8
2,95002,78.216717,Bio Slime,Horror Action Thriller Sci-Fi,en,2010,4.0
3,98251,78.276995,Kabei: Our Mother,Family Drama,ja,2008,7.4
4,111040,78.313534,3 Holiday Tails,Family Comedy Romance,en,2011,4.9
5,92243,78.326706,The Flowers of War,Drama History War,zh,2011,7.4
6,6579,78.544614,"One, Two, Three",Comedy Family,en,1961,7.7
7,100338,78.666779,A Life Apart: Hasidism in America,Documentary Family,en,1998,4.3
8,33292,78.870523,The Human Comedy,Comedy Drama Family,en,1943,6.3
9,44947,79.021956,"Lacombe, Lucien",Drama War,fr,1974,7.1


In [33]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,7702,13.370115,The Bells of St. Mary's,Drama Family,en,1945,7.2
1,2032,13.687145,The Barefoot Executive,Comedy Family,en,1971,5.2
2,56945,13.723250,The Perfect Holiday,Comedy Drama Romance Family,en,2007,5.4
3,121141,13.812691,A Cry in the Wild,Drama Family,en,1990,6.9
4,250,13.974960,Heavyweights,Comedy Family Drama,en,1995,6.0
5,123244,13.986796,"Cry, the Beloved Country",Drama,en,1952,5.5
6,118230,13.996426,Mahoney's Last Stand,Comedy Drama,en,1972,5.3
7,97880,14.169585,A Child Is Waiting,Drama,en,1963,6.9
8,27713,14.216063,Bukowski - Born into This,Documentary,en,2003,7.7
9,90201,14.218497,The Grand Dukes,Comedy,fr,1996,6.3


In [34]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,101222,2.778105,Elevator,Thriller,en,2011,5.0
1,25805,2.849370,L'Atalante,Romance Drama,fr,1934,7.6
2,78517,2.850545,Joan Rivers: A Piece of Work,Documentary,en,2010,6.9
3,54617,2.888053,The Brice Man,Comedy,fr,2005,5.4
4,36477,2.933339,The Baxter,Comedy Romance,en,2005,5.8
5,240,2.936636,Hideaway,Drama Thriller,en,1995,5.1
6,100436,2.944566,The Gatekeepers,Documentary,en,2012,7.2
7,67087,2.957109,"I Love You, Man",Comedy,en,2009,6.5
8,81316,2.962744,Deadline,Drama Horror Thriller,en,2009,5.1
9,63072,2.972014,The Road,Adventure Drama,en,2009,6.9


In [36]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,59382,0.668943,Flight of the Red Balloon,Family Drama,fr,2007,6.0
1,87327,0.706861,Camp de Thiaroye,War Drama,fr,1988,6.5
2,94539,0.707431,Viva Riva!,Action Drama Thriller,fr,2010,5.7
3,56945,0.709111,The Perfect Holiday,Comedy Drama Romance Family,en,2007,5.4
4,33292,0.712633,The Human Comedy,Comedy Drama Family,en,1943,6.3
5,55668,0.713315,Full of It,Comedy Family Drama,en,2007,5.3
6,98251,0.713427,Kabei: Our Mother,Family Drama,ja,2008,7.4
7,26681,0.720316,Bullet in the Head,Thriller Crime Drama,cn,1990,7.8
8,111040,0.721442,3 Holiday Tails,Family Comedy Romance,en,2011,4.9
9,8708,0.723076,Hometown Legend,Comedy Drama Family,en,2002,5.0


In [37]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,56945,119.199844,The Perfect Holiday,Comedy Drama Romance Family,en,2007,5.4
1,7702,120.373260,The Bells of St. Mary's,Drama Family,en,1945,7.2
2,118230,121.110016,Mahoney's Last Stand,Comedy Drama,en,1972,5.3
3,27713,122.230865,Bukowski - Born into This,Documentary,en,2003,7.7
4,2032,124.636742,The Barefoot Executive,Comedy Family,en,1971,5.2
5,250,126.000191,Heavyweights,Comedy Family Drama,en,1995,6.0
6,121141,126.209320,A Cry in the Wild,Drama Family,en,1990,6.9
7,42393,126.281647,Love's Enduring Promise,TV Movie Drama Romance,en,2004,6.8
8,59382,126.541031,Flight of the Red Balloon,Family Drama,fr,2007,6.0
9,111040,126.567200,3 Holiday Tails,Family Comedy Romance,en,2011,4.9


# TD3

In [43]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [44]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,92178,8.885455,Can Go Through Skin,Drama,nl,2009,6.5
1,95967,8.967194,Marriage Retreat,Comedy Drama Romance,en,2011,6.1
2,83976,9.023332,The Trip,Comedy Drama,en,2010,7.0
3,106897,9.131948,Stuck Between Stations,Drama Romance,en,2011,6.2
4,87449,9.250244,Stricken,Drama,nl,2009,6.0
5,1498,9.254573,Inventing the Abbotts,Drama Romance,en,1997,6.1
6,78344,9.320072,The Song of Sparrows,Drama,fa,2008,7.0
7,73526,9.336596,A Matter of Dignity,Drama,el,1958,7.0
8,121821,9.340507,The Dancer,Drama,id,2011,5.9
9,27447,9.360726,Southern Comfort,Documentary Romance,en,2001,7.6


In [45]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,74791,0.657750,A Town Called Panic,Adventure Animation Comedy Family Fantasy,fr,2009,7.6
1,89060,0.670064,Cold Prey,Drama Horror Mystery Thriller,no,2006,6.2
2,70293,0.680508,Julie & Julia,Romance Drama,en,2009,6.6
3,126094,0.681883,Claymation Comedy of Horrors,Animation,en,1991,8.0
4,52199,0.687294,Damnation,Crime Drama Romance Musical Musical,hu,1988,8.0
5,65894,0.691203,Fear(s) of the Dark,Animation Comedy Drama Horror Mystery Romance,fr,2008,6.9
6,80740,0.691300,Luke and Lucy: The Texas Rangers,Animation Adventure Comedy Western,qbn,2009,5.7
7,94659,0.695785,Walking with Prehistoric Beasts,Animation Documentary,en,2001,8.3
8,7238,0.701478,Ashes and Diamonds,War Drama,pl,1958,7.7
9,45533,0.706804,Conspirators of Pleasure,Drama Animation Comedy,cs,1996,7.1


In [41]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,26520,0.672432,Full Moon in Paris,Drama Romance,fr,1984,7.1
1,30810,0.687112,The Life Aquatic with Steve Zissou,Adventure Comedy Drama,en,2004,7.2
2,88740,0.698927,Above Suspicion,Drama Thriller,en,1943,6.0
3,4608,0.699190,An Innocent Man,Action Crime Drama Thriller Comedy,en,1989,6.3
4,107234,0.700899,Deadly Surveillance,TV Movie Thriller Action,en,1991,4.0
5,105928,0.704404,The Day of the Crows,Animation,fr,2012,6.7
6,55116,0.706569,The Hunting Party,Action Adventure Thriller Drama,en,2007,6.6
7,6074,0.709194,Endangered Species,Thriller Drama Mystery,en,1982,6.4
8,97172,0.714985,Frankenweenie,Animation Comedy Family,en,2012,6.8
9,8634,0.715056,Tank,Action Adventure Comedy Drama Thriller,en,1984,5.8


In [42]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,116080,78.378740,Torture Garden,Horror,en,1967,6.3
1,3225,79.868615,Down to You,Comedy Drama Family Romance,en,2000,5.0
2,1565,80.137916,Head Above Water,Comedy Thriller,en,1996,5.9
3,114900,81.220973,Stranger on the Prowl,Drama,en,1952,0.0
4,6986,81.266239,Ben-Hur: A Tale of the Christ,Adventure Drama,en,1925,7.5
5,1943,81.314975,The Greatest Show on Earth,Action Drama Romance Comedy,en,1952,6.4
6,72507,81.455763,Second In Command,Action,en,2006,4.9
7,7822,81.586494,Mogambo,Adventure Drama Romance,en,1953,6.5
8,101592,81.605820,A Bird of the Air,Drama Romance,en,2011,6.0
9,1384,81.627182,The Substance of Fire,Drama,en,1996,5.5


In [43]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,101592,9.037744,A Bird of the Air,Drama Romance,en,2011,6.0
1,72153,9.051922,A Dangerous Woman,Drama Romance,en,1993,5.2
2,80789,9.061975,Loving Leah,Drama Romance TV Movie,en,2009,6.5
3,114017,9.289977,Career,Drama,en,1959,7.0
4,87449,9.408204,Stricken,Drama,nl,2009,6.0
5,83821,9.419411,The Happy Ending,Drama,en,1969,6.3
6,959,9.438744,Of Human Bondage,Drama,en,1934,6.9
7,44731,9.441901,Stay Alive,Horror Thriller,en,2006,5.4
8,106897,9.482531,Stuck Between Stations,Drama Romance,en,2011,6.2
9,6159,9.482532,All the Real Girls,Drama Romance,en,2003,6.1


In [44]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,114017,1.827497,Career,Drama,en,1959,7.0
1,53187,1.872138,Beauty in Trouble,Comedy Drama,cs,2006,6.9
2,6636,1.946254,The Sure Thing,Comedy Drama Romance,en,1985,6.7
3,51535,1.954173,Body Count,Action Thriller,en,1998,5.5
4,27293,1.962612,The Stepdaughter,Horror,en,2000,5.0
5,80701,1.971506,Mudhoney,Drama,en,1965,7.3
6,97163,1.973294,Shameless,Drama,no,2010,6.5
7,6733,1.984957,I'm Going Home,Drama,fr,2001,7.4
8,126981,2.012283,The Wayward Girl,Drama,no,1959,6.5
9,78264,2.026607,The Back-Up Plan,Comedy Romance,en,2010,5.9


In [45]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,26520,0.712573,Full Moon in Paris,Drama Romance,fr,1984,7.1
1,4608,0.715002,An Innocent Man,Action Crime Drama Thriller Comedy,en,1989,6.3
2,55116,0.729309,The Hunting Party,Action Adventure Thriller Drama,en,2007,6.6
3,4571,0.741175,Bill & Ted's Excellent Adventure,Adventure Comedy Sci-Fi,en,1989,6.8
4,30810,0.745511,The Life Aquatic with Steve Zissou,Adventure Comedy Drama,en,2004,7.2
5,82848,0.745768,One Week,Comedy,en,1920,7.9
6,6072,0.746171,The Unfaithful Wife,Drama Thriller,fr,1969,7.0
7,6074,0.751405,Endangered Species,Thriller Drama Mystery,en,1982,6.4
8,107234,0.754308,Deadly Surveillance,TV Movie Thriller Action,en,1991,4.0
9,1615,0.755034,The Edge,Action Adventure Drama,en,1997,6.7


In [46]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,101592,76.155228,A Bird of the Air,Drama Romance,en,2011,6.0
1,80789,79.423164,Loving Leah,Drama Romance TV Movie,en,2009,6.5
2,72153,80.107391,A Dangerous Woman,Drama Romance,en,1993,5.2
3,4934,82.907562,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
4,87449,83.086853,Stricken,Drama,nl,2009,6.0
5,75927,83.786263,Haunted Echoes,Horror Thriller,en,2008,4.3
6,82848,83.952698,One Week,Comedy,en,1920,7.9
7,2356,83.958115,Celebrity,Drama Comedy,en,1998,6.0
8,102167,84.406227,Standing Up,Drama,en,2013,6.8
9,104774,84.677429,Three Worlds,Drama,fr,2012,4.9
